# Project: Wrangling and Analyze Data

In [1]:
import pandas as pd
import numpy as np
import json
import requests as r

## Data Gathering

1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
#Download and import the dataset from twitter_archive.csv into pandas DataFrame
twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
twitter_archive_df.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1614,685307451701334016,NaN,NaN,2016-01-08 03:50:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Petrick. He's an Altostratus Floo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/685307451...,11,10,Petrick,None,floofer,None,None


In [4]:
# Set the column widths wider to enable a better view of truncated texts
pd.set_option('display.max_colwidth', 350)

In [5]:
# Inspect and investigate the dataset programmatically
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [6]:
# What are the unique values in the rating_numerator column
twitter_archive_df.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2])

In [7]:
#Number of records with denominator values that are not 10
len(twitter_archive_df[twitter_archive_df['rating_denominator']!=10])

23

In [8]:
#count the number of duplicates in the dataframe
sum(twitter_archive_df.tweet_id.duplicated())

0

#### 2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [9]:
image_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# Send a get request to the server that host the image prediction url and save
# the response in a variable called response.
response = r.get(image_url)

# Use a context manager to open the file and write the response content in a
# new file called img_url_data.tsv
with open('img_url_data.tsv','wb') as file:
    file.write(response.content)
    
# Read the saved content into a pandas DataFrame
img_url_df = pd.read_csv('img_url_data.tsv', sep='\t')
img_url_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [10]:
#How many unique id(s) do we have in this file
img_url_df.tweet_id.nunique()

2075

In [11]:
# Are there any duplicates
sum(img_url_df.tweet_id.duplicated())

0

In [12]:
# Investigate the dataset on a broader view
img_url_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


#### 3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [13]:
# Create an empty list to hold our data
json_data = []


# Use a context manager to open the file and read the response content in a
# new file called tweet-json.txt.
with open('tweet-json.txt', mode='r') as file:
    #loop through the file and read each line in the tweet-json.txt file
    for line in file:
        #load the lines into the data variable line by line
        data = json.loads(line)
        
        #Append each line into the json_data empty list object
        json_data.append({'tweet_id': data['id'],
                        'retweet_count': data['retweet_count'],
                        'favorite_count': data['favorite_count']})

#create a pandas dataframe from the json_data list object
tweet_json_df = pd.DataFrame(json_data)
tweet_json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


## Assessing Data




### Quality issues


1. There are Retweeted records in the `text` column of the tweeter_archive_df which indicates duplicates


2. The `source` column contain unnecessary anchor tags


3. `Timestamp` column is an object datatype instead of a datetime datatype. Also, the source column is best converted to a categorical datatype rather than the object datatype


4. Some dog names start with articles such as a, an, actually, one, all. Most of these names starting with lower cases are most likely quality issues. The one is believed to be a None


5. In the `img_url_df`, each image has 3 values of dog breed and of which only one is valid. This has to be addressed.  


6. Some `tweet_id`(s) of the `tweet_archive_df` are missing in the `img_url_df`



7. Some `ratings` have abnormal values in either the `rating_numerator` or
  `rating_denomenator` column



8. Some unnecesary columnn such as `retweeted_status_id`, `retweeted_status_user_id`,	`retweeted_status_timestamp` and `expanded_urls` from the tweeter_archive_df and img_ur_df]



### Tidiness issues

1. `doggo`, `floofer`, `pupper` and `puppo` columns in `twitter_archive_df` table should be merged into one column named "type"

    
2. Merge the `twitter_archive_df` and `img_url_df` and `tweet_json` tables

## Cleaning Data


In [14]:
# Make copies of original pieces of data
twitter_archive_cp = twitter_archive_df.copy()
img_url_cp = img_url_df.copy()
tweet_json_cp = tweet_json_df.copy()

### 1- There are Retweeted records in the text column of the tweeter_archive_df which indicates duplicates

- remove all records that have `retweeted_status_id` values

#### Code

In [15]:
# count the sum of recorded that are retweets.  
sum(twitter_archive_cp['retweeted_status_id'].notna())

181

In [16]:
# Filter out the unwanted records, i.e (retweets)
twitter_archive_cp = (
                    twitter_archive_cp[twitter_archive_cp
                                       ['retweeted_status_id']
                                       .isna()
                                      ]
                     )

twitter_archive_cp.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


#### Test

In [17]:
# Confirm if there are still any retweeted records
sum(twitter_archive_cp['retweeted_status_id'].notna())

0

### 2- The source column contain unnecessary anchor tags

- strip away all anchor tags and their attributes and leave just the text between the opening and closing tags

#### Code

In [18]:
# Use a regular expression to get rid of the patterns that are not needed. 
# That means only the text content is what we are interested in
twitter_archive_cp['source'] = (twitter_archive_cp['source']
                                .str.replace(r'<(?:a\b[^>]*>|/a>)','')
                               )

twitter_archive_cp['source'].replace("Twitter for","")
twitter_archive_cp['source'].head()

0    Twitter for iPhone
1    Twitter for iPhone
2    Twitter for iPhone
3    Twitter for iPhone
4    Twitter for iPhone
Name: source, dtype: object

#### Test

In [19]:
# make a unique value counts of all unique values in the column
twitter_archive_cp['source'].value_counts()

Twitter for iPhone     2042
Vine - Make a Scene      91
Twitter Web Client       31
TweetDeck                11
Name: source, dtype: int64

### 3- Timestamp column is an object datatype instead of a datetime datatype. Also, the source column is best converted to a categorical datatype rather than the object datatype

 - Convert the `Timestamp` column to a datetime and `source` column to categorical datatype

### code

In [20]:
#convert the Timestamp to datetime
twitter_archive_cp['timestamp'] = (pd.to_datetime(twitter_archive_cp
                                                  ['timestamp'])
                                  )

#test this column validity
twitter_archive_cp['timestamp'].head()

0   2017-08-01 16:23:56
1   2017-08-01 00:17:27
2   2017-07-31 00:18:03
3   2017-07-30 15:58:51
4   2017-07-29 16:00:24
Name: timestamp, dtype: datetime64[ns]

In [21]:
# Convert this column to a categorical datatype
twitter_archive_cp['source'] = twitter_archive_cp['source'].astype('category')

# Test the datatype validity of this column now
twitter_archive_cp['source'].dtype

CategoricalDtype(categories=['TweetDeck', 'Twitter Web Client', 'Twitter for iPhone',
                  'Vine - Make a Scene'],
                 ordered=False)

### 4- Some dog names start with articles such as a, an, actually, one, all. Most of these names starting with lower cases are most likely quality issues. The one is believed to be a None

#### Define
- Convert all lowercased letters to None. These are believed not to be actual
  dog names

#### Code

In [22]:
# Match all dog names that beging with lowecases or articles included. These
# probably aren't names

twitter_archive_cp[twitter_archive_cp['name'].str.match(r'([a-z])')] 

# Convert these names to None
twitter_archive_cp.name[twitter_archive_cp['name'].str.match(r'([a-z])')]='None'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


#### Test

In [23]:
# Confirm if there are no more lowecase or article names
twitter_archive_cp[twitter_archive_cp['name'].str.match(r'([a-z])')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### 5- In the `img_url_df`, each image has 3 values of dog breed and of which only one is valid. This has to be addressed.

#### Define

- Clean the img_url_cp data, such that, every record has one and only one dog breed and one corresponding confidence value by combining the following columns `p1	p1_conf	p1_dog	p2	p2_conf	p2_dog	p3	p3_conf	p3_dog`

In [24]:
#Create a selection condition 
condition = [(img_url_cp['p1_dog'] == True), (img_url_cp['p2_dog'] == True), (img_url_cp['p3_dog'] == True)]

# Combine all dog breeds columns into a single list
breed = [img_url_cp['p1'], img_url_cp['p2'], img_url_cp['p3']]

# Also, combine all dog breed confidence rating by the algorith into a single
# column
confidence = [img_url_cp['p1_conf'], img_url_cp['p2_conf'], img_url_cp['p3_conf']]


# Using numpy's np.select(), create a new column for both breed and confidence
# by using the boolean condition column
img_url_cp['breed'] = np.select(condition, breed, default ='none')
img_url_cp['confidence'] = np.select(condition, confidence, default =0)

In [25]:
img_url_cp.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,breed,confidence
1651,809448704142938112,https://pbs.twimg.com/media/Czu9RiwVEAA_Okk.jpg,1,Greater_Swiss_Mountain_dog,0.375415,True,Cardigan,0.134317,True,English_springer,0.073697,True,Greater_Swiss_Mountain_dog,0.375415
909,700518061187723268,https://pbs.twimg.com/media/Cbi9dI_UYAAgkyC.jpg,1,American_Staffordshire_terrier,0.569501,True,Staffordshire_bullterrier,0.211308,True,Chihuahua,0.121839,True,American_Staffordshire_terrier,0.569501
458,674788554665512960,https://pbs.twimg.com/media/CV1Um8vWIAAmhQn.jpg,1,miniature_poodle,0.349561,True,toy_poodle,0.154711,True,Maltese_dog,0.134229,True,miniature_poodle,0.349561
41,666701168228331520,https://pbs.twimg.com/media/CUCZLHlUAAAeAig.jpg,1,Labrador_retriever,0.887707,True,Chihuahua,0.029307,True,French_bulldog,0.020756,True,Labrador_retriever,0.887707
201,669682095984410625,https://pbs.twimg.com/media/CUswUBRUAAAahAo.jpg,1,Christmas_stocking,0.188397,False,studio_couch,0.086887,False,bookcase,0.082599,False,none,0.000000


### 6- Since we have to use the `img_url_cp` and `twitter_archive_cp` together, we need to ensure we keep those records that are missing in the twitter_archive_cp are taken out

#### Define
- Filter the rows in the twitter_archive_cp by only those records that is both in it and the img_url_cp tables

#### Code

In [26]:
# How many records do we have in the img_url_cp table
img_url_cp.tweet_id.count()

2075

In [27]:
# How many records are there in  the twitter_archive_cp table
twitter_archive_cp.tweet_id.count()

2175

In [28]:
# How many records are presents in both tables
sum(img_url_cp.tweet_id.isin(twitter_archive_cp.tweet_id))

1994

In [29]:
# Now we have to select only the 1994 records that exist in the 2 tables
twitter_archive_cp = twitter_archive_cp[twitter_archive_cp.tweet_id.isin(img_url_cp.tweet_id)]

#### Test

In [30]:
len(twitter_archive_cp[~twitter_archive_cp.tweet_id.isin(img_url_cp.tweet_id)])

0

### 7- Some `ratings` have abnormal values in either the `rating_numerator` or `rating_denomenator` column

#### Define
- Some `ratings` have abnormal values in either the `rating_numerator` or
  `rating_denomenator` column. These somehow affects ratings

#### Code

In [31]:
twitter_archive_cp.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


In [32]:
# Create a ratings column by dividing the rating_numerator by 
# the rating_denominator
twitter_archive_cp['rating'] = (twitter_archive_cp.rating_numerator/
                                    twitter_archive_cp.rating_denominator
                                )

twitter_archive_cp.rating_denominator.value_counts()

10     1976
50        3
80        2
11        2
170       1
150       1
130       1
120       1
110       1
90        1
70        1
40        1
20        1
7         1
2         1
Name: rating_denominator, dtype: int64

In [33]:
# What are the largest and smallest values in the rating_numerator column
display(twitter_archive_cp.rating_numerator.max(),twitter_archive_cp.rating_numerator.min())

1776

0

In [34]:
# What are the largest and smallest values in the rating_denominator column
display(twitter_archive_cp.rating_denominator.max(),twitter_archive_cp.rating_denominator.min())

170

2

In [35]:
#Because these values are way too small or big in comparison with other values,
#it will seem appropriate to filter them out

twitter_archive_cp = twitter_archive_cp[~((twitter_archive_cp['rating_denominator']==twitter_archive_cp.rating_denominator.min()) | (twitter_archive_cp['rating_denominator']==twitter_archive_cp.rating_denominator.max()))]


In [36]:
twitter_archive_cp.rating.describe()

count    1992.000000
mean        1.169570
std         4.069085
min         0.000000
25%         1.000000
50%         1.100000
75%         1.200000
max       177.600000
Name: rating, dtype: float64

In [37]:
#We also have abnormally high and low rating values that are worth filtering 
# out

twitter_archive_cp = twitter_archive_cp[~((twitter_archive_cp['rating']==twitter_archive_cp.rating.min()) | (twitter_archive_cp['rating']==twitter_archive_cp.rating.max()))]


In [38]:
#twitter_archive_cp.info()

#### Test

In [39]:
twitter_archive_cp.rating.describe()

count    1989.000000
mean        1.082043
std         0.956873
min         0.100000
25%         1.000000
50%         1.100000
75%         1.200000
max        42.000000
Name: rating, dtype: float64

In [40]:
#Set a quantile of 0.1, to use only values greater than 0.3 ratings, anything 
# else lower than 0.1 is rather too low
x = twitter_archive_cp.rating.quantile(.01)


# Set a quantile of .99 to use values below 99% of the rating values
y = twitter_archive_cp.rating.quantile(.99)


# Filter the dataset based on the upper and lower boundries of the quantiles
twitter_archive_cp = twitter_archive_cp[~((twitter_archive_cp.rating > y) | (twitter_archive_cp.rating < x))]
twitter_archive_cp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1970 entries, 0 to 2355
Data columns (total 18 columns):
tweet_id                      1970 non-null int64
in_reply_to_status_id         22 non-null float64
in_reply_to_user_id           22 non-null float64
timestamp                     1970 non-null datetime64[ns]
source                        1970 non-null category
text                          1970 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1970 non-null object
rating_numerator              1970 non-null int64
rating_denominator            1970 non-null int64
name                          1970 non-null object
doggo                         1970 non-null object
floofer                       1970 non-null object
pupper                        1970 non-null object
puppo                         1970 non-null object
rating                      

In [41]:
twitter_archive_cp.rating.describe()

count    1970.000000
mean        1.061703
std         0.203917
min         0.300000
25%         1.000000
50%         1.100000
75%         1.200000
max         1.400000
Name: rating, dtype: float64

### 8- Some unnecesary columnn such as `retweeted_status_id`, `retweeted_status_user_id`,	`retweeted_status_timestamp` and `expanded_urls` from the tweeter_archive_df and img_ur_df]

#### Define
- Remove the columns that have no use for the sake of our analysis. These columns are either not useful or carry very minimal effect on our data, so they can be dropped

#### Code

In [42]:
#select the columns to delete in the twitter_archive_cp DataFrame
drop_cols1 = ['retweeted_status_id','retweeted_status_user_id',
              'retweeted_status_timestamp','expanded_urls',
             'in_reply_to_status_id','in_reply_to_user_id','rating_numerator',
             'rating_denominator']

#select the columns to delete in the img_url_cp DataFrame
drop_cols_in_img_df = ['img_num','p1','p1_conf','p1_dog','p2',
                       'p2_conf','p2_dog','p3','p3_conf','p3_dog']


# Drop these 
twitter_archive_cp.drop(columns=drop_cols1, inplace=True)

img_url_cp.drop(columns=drop_cols_in_img_df, inplace=True)

#### Test

In [43]:
twitter_archive_cp.sample()

,tweet_id,timestamp,source,text,name,doggo,floofer,pupper,puppo,rating
709,785170936622350336,2016-10-09 17:31:53,Twitter for iPhone,This is Hero. He was enjoying the car ride until he remembered that bees are dying globally at an alarming rate. 11/10 https://t.co/cubFg7F4qQ,Hero,None,None,None,None,1.1


In [44]:
img_url_cp.sample()

,tweet_id,jpg_url,breed,confidence
1336,758355060040593408,https://pbs.twimg.com/media/CoY324eWYAEiDOG.jpg,Pembroke,0.987643


### *Quality Issue 1*- doggo, floofer, pupper and puppo columns in twitter_archive_df table should be merged into one column named stages

#### Define
- The dog stages are divided into 4 columns and most records have None dog_stages. These need to be melted into a single column we can call dog_stage

#### Code

In [45]:
# List the dogs stages columns in a list
stages =[ 'doggo', 'floofer', 'pupper', 'puppo']


# Replace all None values with equivalent np.nan
twitter_archive_cp[stages] = twitter_archive_cp[stages].replace('None', np.nan)

In [46]:
twitter_archive_cp.sample()

,tweet_id,timestamp,source,text,name,doggo,floofer,pupper,puppo,rating
1564,688116655151435777,2016-01-15 21:52:49,Twitter for iPhone,Please send dogs. I'm tired of seeing other stuff like this dangerous pirate. We only rate dogs. Thank you... 10/10 https://t.co/YdLytdZOqv,None,NaN,NaN,NaN,NaN,1.0


In [ ]:
# Join the valid dog stages that are left together with commas,
def join(valid_list):
    
    #Drop the np.nan values that we now have and convert the rest of the values 
    # to string
    return ', '.join(valid_list.dropna().astype(str)) 


#Call the join function on the dog stages column and subsequently create a new
#column from the result
twitter_archive_cp['dog_stage'] = twitter_archive_cp[stages].apply(join, axis=1) 

In [ ]:
# Replace all remaining np.nan with spaces
twitter_archive_cp['dog_stage'] = twitter_archive_cp['dog_stage'].replace('', np.nan)

In [ ]:
# we can go ahead and drop the initial stages list we defined earlier
twitter_archive_cp.drop(columns=stages, inplace=True)

In [ ]:
twitter_archive_cp.sample()

#### Test

In [ ]:
twitter_archive_cp.dog_stage.value_counts()

### *Quality Issue 2* - Merge the twitter_archive_cp and img_url_cp tables

#### Define
- Merge all datasets on the same tweet_id. We use the twitter_archive_cp as the left most data because, this is the main dataset with the original tweets while the other tables are supporting tables with additional information 

#### Code

In [ ]:
#Combine the 3 datasets together into one where there are unique tweet_id in 
# all datasets
combined_df = twitter_archive_cp.merge(img_url_cp, how='left', on='tweet_id')\
                .merge(tweet_json_cp, how='left', on='tweet_id')



#### Test

In [ ]:
combined_df.sample()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
combined_df.to_csv("twitter_archive_master.csv", index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

In [ ]:
twitter_archive_master = pd.read_csv("twitter_archive_master.csv")
twitter_archive_master.head()

### Questions to ask

- Which dog got the highest ratings

- What is the most popular source of tweets

- Which the most liked dog breed

#### Question 1

In [ ]:
# What is the highest rated dog and most liked dog
highest_rated_dogs = twitter_archive_master[twitter_archive_master['rating']== twitter_archive_master['rating'].max()]

highest_rated_dogs.groupby('name').agg({'favorite_count':'sum','retweet_count':
                                        'sum'}).sort_values(by=['favorite_count'],ascending=False)[1:3]


- There were many dogs with the highest rating of 1.4. Most of them have no name, however, among those that have names, **Bo** is the dog with the most likes

#### Question 2

In [ ]:
# Import matplotlib inline to display the chart in the notebook
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# Group by the source and plot a chart to view the result
twitter_archive_master.groupby('source').agg({'source':'count'})\
                                        .plot(kind='barh',log=True);

plt.xlabel("Number of Tweets", fontsize=14);
plt.ylabel('Tweet Source', fontsize=14);
plt.title('Number of Tweets by Source', fontsize=14,weight='bold');
plt.savefig('source_fig')

In [ ]:
#Total Number of records
totalNumber = len(twitter_archive_master)

# Total records for tweets from iphones
iphoneRec = len(twitter_archive_master[twitter_archive_master['source']==
                                       'Twitter for iPhone'
                                      ])

# What is the percentage records of iphones
int(round(iphoneRec/totalNumber,2)*100)

- The iPhone device has recorded the highest number of tweets in the datasets with a whooping 98% of the grand total

#### Question 3

In [ ]:
# What is the most liked dog breed
highest_rated_breed = twitter_archive_master[twitter_archive_master['rating']== twitter_archive_master['rating'].max()]

highest_rated_breed.groupby('breed',).agg({'favorite_count':'sum','retweet_count':
                                        'sum'}).sort_values(by=['favorite_count'],ascending=False)[:3]

total = twitter_archive_master.favorite_count.sum()
French_BullDog = twitter_archive_master[twitter_archive_master.breed == 'French_bulldog']['favorite_count'].sum()
round((French_BullDog/total),2)*100

- Amongst all the dog breeds whose records were provided, the French_bulldog is the most liked

### Insights:
1. There were many dogs with the highest rating of 1.4. Most of them have no name, however, among those that have names, **Bo** is the most liked dog


2. The iPhone device has recorded the highest number of tweets in the datasets with a whooping 98% of the grand total


3. Amongst all the dog breeds whose records were provided, the **French_bulldog** is the most liked

### Visualization

In [ ]:
# What is the relationship between retweets and favorite counts
sns.regplot('favorite_count','retweet_count',data=twitter_archive_master);

plt.xlabel("Favorite Counts", fontsize=14);
plt.ylabel("Retweet Counts", fontsize=14);
plt.title("Favorite Counts vs Retweet Counts", fontsize=14 ,weight='bold');
plt.savefig('tweet_RT_fig')

In [ ]:
twitter_archive_master.groupby('name').agg({'name':'count'})\
                            .sort_values(by='name',ascending=False)[1:11]\
                            .plot(kind='bar');

plt.xlabel("Dog Name", fontsize=14);
plt.ylabel("Name Counts", fontsize=14);
plt.title("Dog Name vs Name Counts", fontsize=14 ,weight='bold');
plt.savefig('dog_names_fig')

In [ ]:
#Top 5 breeds by mean ratings
twitter_archive_master.loc[1:,['breed','rating']].groupby('breed').mean()\
                            .sort_values(by='rating', ascending=False)[:5]\
                            .plot(kind='bar');

plt.xlabel("Breed", fontsize=14)
plt.ylabel('Rating', fontsize=14)
plt.title('Top 5 Dog Breeds by Average Ratings', fontsize=14,weight='bold');
plt.savefig('top5_fig')

In [ ]:
#Average Dog rating distribution
twitter_archive_master.loc[1:,['breed','rating']].groupby('breed').mean().hist(grid=False);

plt.xlabel("Ratings", fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('Distibution of Average Dog Ratings',weight='bold', fontsize=14);
plt.savefig('top5_fig')